In [ ]:
! cat ../../.keras/keras.json

In [ ]:
import gc
!python --version

In [ ]:
import numpy as np
import cv2
from tqdm import tqdm

from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Input
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.optimizers import Adam, SGD
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from keras import applications

from sklearn.metrics import fbeta_score
# from sklearn.metrics import matthews_corrcoef, hamming_loss

In [ ]:
RESULTS_DIR = "./results/session-01-KagglePlanetAmazon/"
DATA_DIR = "../../Data/kaggle-planet-amazon/"

# !head ../../Data/kaggle-planet-amazon/train_v2.csv

import pandas as pd

df_train = pd.read_csv(DATA_DIR + "train_v2.csv")

# df_train["tags"] = df_train["tags"].map(lambda x: x.split(" "))

# df_train.head()

# binarizedLabels_df = pd.DataFrame(binarizedLabels)
# binarizedLabels_df.columns = mlb.classes_
# binarizedLabels_df.head()

# df_valid_samples = df_train.sample(frac=0.2)

# df_train_samples = df_train.drop(df_valid_samples.index)

# def PrintShape(df, df_shape):
#     print("{} shape: {}".format(df, df_shape))

# PrintShape('df_train_samples', df_train_samples.shape)
# PrintShape('df_valid_samples', df_valid_samples.shape)

In [ ]:
# from sklearn.preprocessing import MultiLabelBinarizer

# mlb = MultiLabelBinarizer()

# y_train = mlb.fit_transform(df_train_samples["tags"])
# y_valid = mlb.fit_transform(df_valid_samples["tags"])

# mlb.classes_

####################################

flatten = lambda l: [item for sublist in l for item in sublist]
labels = list(set(flatten([l.split(' ') for l in df_train['tags'].values])))

label_map = {l: i for i, l in enumerate(labels)}
inv_label_map = {i: l for l, i in label_map.items()}

# Params
input_size = 64
input_channels = 3

epochs = 5
batch_size = 128
learning_rate = 0.001
lr_decay = 1e-4

In [ ]:
# x_valid = []
# y_valid = []

# for f, tags in tqdm(df_valid_samples.values, miniters=1000):
#     img = cv2.resize(cv2.imread(DATA_DIR + 'train-jpg/{}.jpg'.format(f)), (input_size, input_size))
#     targets = np.zeros(17)
#     for t in tags.split(' '):
#         targets[label_map[t]] = 1
#     x_valid.append(img)
#     y_valid.append(targets)

# y_valid = np.array(y_valid, np.uint8)
# x_valid = np.array(x_valid, np.float32)

# x_train = []
# y_train = []

# for f, tags in tqdm(df_train_samples.values, miniters=1000):
#     img = cv2.resize(cv2.imread(DATA_DIR + 'train-jpg/{}.jpg'.format(f)), (input_size, input_size))
#     targets = np.zeros(17)
#     for t in tags.split(' '):
#         targets[label_map[t]] = 1
#     x_train.append(img)
#     y_train.append(targets)
#     img = cv2.flip(img, 0)  # flip vertically
#     x_train.append(img)
#     y_train.append(targets)
#     img = cv2.flip(img, 1)  # flip horizontally
#     x_train.append(img)
#     y_train.append(targets)
#     img = cv2.flip(img, 0)  # flip vertically
#     x_train.append(img)
#     y_train.append(targets)

# y_train = np.array(y_train, np.uint8)
# x_train = np.array(x_train, np.float32)

df_test_samples = pd.read_csv(DATA_DIR + 'sample_submission_v2.csv')

# x_test = []

# for f, tags in tqdm(df_test_samples.values, miniters=1000):
#     img = cv2.resize(cv2.imread(DATA_DIR + 'test-jpg/{}.jpg'.format(f)), (input_size, input_size))
#     x_test.append(img)

# x_test = np.array(x_test, np.float32)

In [ ]:
import pickle

# with open('image_vars.pkl', 'wb') as opfile:
#     pickle.dump(y_valid, opfile, protocol=4)
#     pickle.dump(x_valid, opfile, protocol=4)
#     pickle.dump(y_train, opfile, protocol=4)
#     pickle.dump(x_train, opfile, protocol=4)
#     pickle.dump(x_test, opfile, protocol=4)

y_valid = []
x_valid = []
y_train = []
x_train = []
x_test = []

with open('image_vars.pkl', 'rb') as ipfile:
    y_valid = pickle.load(ipfile)
    x_valid = pickle.load(ipfile)
    y_train = pickle.load(ipfile)
    x_train = pickle.load(ipfile)
    x_test = pickle.load(ipfile)

In [ ]:
# using functional API

from keras.layers import Input, Dense
from keras.models import Model

# This returns a tensor
inputs = Input(shape=(input_size, input_size, input_channels))

# a layer instance is callable on a tensor, and returns a tensor
x = BatchNormalization()(inputs)
x = Conv2D(32, kernel_size=(3, 3), padding='same', activation='relu')(x)
x = Conv2D(32, kernel_size=(3, 3), activation='relu') (x)
x = MaxPooling2D(pool_size=(3, 3))(x)
x = Dropout(0.25)(x)

x = BatchNormalization()(x)
x = Conv2D(64, kernel_size=(3, 3), padding='same', activation='relu')(x)
x = Conv2D(64, kernel_size=(3, 3), activation='relu')(x)
x = MaxPooling2D(pool_size=(3, 3))(x)
x = Dropout(0.25)(x)

x = BatchNormalization()(x)
x = Conv2D(128, kernel_size=(3, 3), padding='same', activation='relu')(x)
x = Conv2D(128, kernel_size=(3, 3), activation='relu')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Dropout(0.25)(x)

x = BatchNormalization()(x)
x = Conv2D(256, kernel_size=(3, 3), padding='same', activation='relu')(x)
x = Conv2D(256, kernel_size=(2, 2), activation='relu')(x)
# model.add(MaxPooling2D(pool_size=(2, 2)))
x = Dropout(0.25)(x)

x = Flatten()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(17, activation='sigmoid')(x)

# This creates a model that includes
# the Input layer and three Dense layers
model = Model(inputs=inputs, outputs=predictions)

model.summary()

In [ ]:
# # using sequential API

# model = Sequential()
# model.add(BatchNormalization(input_shape=(input_size, input_size, input_channels)))
# model.add(Conv2D(32, kernel_size=(3, 3), padding='same', activation='relu'))
# model.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))
# model.add(MaxPooling2D(pool_size=(3, 3)))
# model.add(Dropout(0.25))

# model.add(BatchNormalization())
# model.add(Conv2D(64, kernel_size=(3, 3), padding='same', activation='relu'))
# model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
# model.add(MaxPooling2D(pool_size=(3, 3)))
# model.add(Dropout(0.25))

# model.add(BatchNormalization())
# model.add(Conv2D(128, kernel_size=(3, 3), padding='same', activation='relu'))
# model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.25))

# model.add(BatchNormalization())
# model.add(Conv2D(256, kernel_size=(3, 3), padding='same', activation='relu'))
# model.add(Conv2D(256, kernel_size=(2, 2), activation='relu'))
# # model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.25))

# model.add(Flatten())
# model.add(Dense(512, activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(17, activation='sigmoid'))

# model.summary()

In [ ]:
callbacks = [EarlyStopping(monitor='val_loss',
                           patience=5,
                           verbose=0),
             TensorBoard(log_dir='logs'),
             ModelCheckpoint('weights.h5',
                             save_best_only=True)]

opt = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

model.compile(loss='binary_crossentropy',
              # We NEED binary here, since categorical_crossentropy l1 norms the output before calculating loss.
              optimizer=opt,
              metrics=['accuracy'])

model.fit(x_train,
          y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=2,
          callbacks=callbacks,
          validation_data=(x_valid, y_valid))  # starts training

In [ ]:
p_test = model.predict(x_test, batch_size=batch_size, verbose=2)
p_test

In [ ]:
p_valid = model.predict(x_valid, batch_size=batch_size)
p_valid

In [ ]:
p_valid = model.predict(x_valid, batch_size=batch_size)
print(fbeta_score(y_valid, np.array(p_valid) >= 0.2, beta=2, average='samples'))

y_test = []

p_test = model.predict(x_test, batch_size=batch_size, verbose=2)
y_test.append(p_test)

result = np.array(y_test[0])
result = pd.DataFrame(result, columns=labels)

preds = []

for i in tqdm(range(result.shape[0]), miniters=1000):
    a = result.ix[[i]]
    a = a.apply(lambda x: x >= 0.2, axis=1)
    a = a.transpose()
    a = a.loc[a[i] == True]
    ' '.join(list(a.index))
    preds.append(' '.join(list(a.index)))

df_test_samples['tags'] = preds
df_test_samples.to_csv('submission.csv', index=False)

In [ ]:
!head submission.csv

In [ ]:
from IPython.display import Image, display
submission_samples = pd.read_csv('./submission.csv')
for f, tags in tqdm(df_valid_samples[:50].values, miniters=1000):
    display(Image(filename=DATA_DIR + 'train-jpg/{}.jpg'.format(f)))
    print(tags)
    print("----------")

In [ ]:
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import hamming_loss

threshold = np.arange(0.1,0.9,0.1)

acc = []
accuracies = []
best_threshold = np.zeros(p_valid.shape[1])
for i in range(p_valid.shape[1]):
    y_prob = np.array(p_valid[:,i])
    for j in threshold:
        y_pred = [1 if prob>=j else 0 for prob in y_prob]
        acc.append(matthews_corrcoef(y_valid[:,i],y_pred))
    acc   = np.array(acc)
    index = np.where(acc==acc.max()) 
    accuracies.append(acc.max()) 
    best_threshold[i] = threshold[index[0][0]]
    acc = []
    
best_threshold

In [ ]:
p_valid = model.predict(x_valid, batch_size=batch_size)
print(fbeta_score(y_valid, np.array(p_valid) >= best_threshold, beta=2, average='samples'))

In [ ]:
# best_threshold

# p_test.shape[1]

# y_pred = np.array([[1 if p_test[i,j]>=best_threshold[j] else 0 for j in range(p_test.shape[1])] for i in range(p_test.shape[0])])

# y_pred

# df_test_samples['tags'] = y_pred
# df_test_samples.to_csv('submission.csv', index=False)

# p_valid = model.predict(x_valid, batch_size=batch_size)
# print(fbeta_score(y_valid, np.array(p_valid) >= 0.2, beta=2, average='samples'))

# y_test = []

# p_test = model.predict(x_test, batch_size=batch_size, verbose=2)
# y_test.append(p_test)

# result = np.array(y_test[0])
# result = pd.DataFrame(result, columns=labels)

# preds = []

# for i in tqdm(range(result.shape[0]), miniters=1000):
#     a = result.ix[[i]]
#     a = a.apply(lambda x: x >= 0.3, axis=1)
#     a = a.transpose()
#     a = a.loc[a[i] == True]
#     ' '.join(list(a.index))
#     preds.append(' '.join(list(a.index)))

# df_test_samples['tags'] = preds
# df_test_samples.to_csv('submission.csv', index=False)

# !head submission.csv

# p_valid = model.predict(x_valid, batch_size=batch_size)
# print(fbeta_score(y_valid, np.array(p_valid) >= 0.3, beta=2, average='samples'))

# y_test = np.array(y_test, np.uint8)

# y_test

# p_valid

# p_test

# from sklearn.metrics import matthews_corrcoef
# from sklearn.metrics import hamming_loss

# threshold = np.arange(0.1,0.9,0.1)

# acc = []
# accuracies = []
# best_threshold = np.zeros(p_valid.shape[1])
# for i in range(p_valid.shape[1]):
#     y_prob = np.array(p_valid[:,i])
#     for j in threshold:
#         y_pred = [1 if prob>=j else 0 for prob in y_prob]
#         acc.append(matthews_corrcoef(y_valid[:,i],y_pred))
#     acc   = np.array(acc)
#     index = np.where(acc==acc.max()) 
#     accuracies.append(acc.max()) 
#     best_threshold[i] = threshold[index[0][0]]
#     acc = []

# best_threshold

# print(fbeta_score(y_valid, np.array(p_valid) > 0.4, beta=2, average='samples'))

# len(y_test[0])

# y_pred = np.array([[1 if p_valid[i,j]>=best_threshold[j] else 0 for j in range(y_valid.shape[1])] for i in range(len(y_test[0]))])

# hamming_loss(y_valid,y_pred)

# total_correctly_predicted = len([i for i in range(len(y_valid)) if (y_valid[i]==y_pred[i]).sum() == 17])

# total_correctly_predicted

# x_valid.shape

# (total_correctly_predicted/ x_valid.shape[0]) * 100

In [ ]:
gc.collect()

In [ ]:
def save_bottleneck_features():
    model = applications.VGG16(weights = 'imagenet', include_top = False)

    datagen = ImageDataGenerator(
                    rescale=1./255,
                    rotation_range = 40,
                    width_shift_range = 0.2,
                    height_shift_range = 0.2,
                    shear_range = 0.2,
                    zoom_range = 0.2,
                    fill_mode = 'nearest'
                    )

    generator = datagen.flow_from_directory(
                            '../../Data/kaggle-planet-amazon/train-jpg/',
                            target_size = (64, 64),
                            batch_size = 128,
                            class_mode = None,
                            shuffle = False
                            )

    bottleneck_features_train = model.predict_generator(generator, len(train_x)//batch_size)
    np.save(open('bottleneck_features_train.npy','wb'), bottleneck_features_train)

    generator = datagen.flow_from_directory(
                            './data/validation',
                            target_size = (img_width, img_height),
                            batch_size = batch_size,
                            class_mode = None,
                            shuffle = False
                            )
    bottleneck_features_val = model.predict_generator(generator, len(val_x)//batch_size)
    np.save(open('bottleneck_features_val.npy','wb'), bottleneck_features_v

In [ ]:
# Current Mean F2-Score: 0.58